### HW.0: KNN
1. Распознавание растровых grayscale изображений через KNN. Равномерное и неравномерное зашумление, сравнить процент для разных k. Сравнить разные меры близости.
2. Добавить выбросы типа 0. Тип 1 - не распознаются KNN, тип 2 - не распознаются ODIN. Сравнить результаты.

In [74]:
from math import sqrt


def euclidean_squared(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return distance

def euclidean_distance(row1, row2):
    return sqrt(euclidean_squared(row1, row2))

def metric_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += abs(row1[i] - row2[i])
    return distance

def cheb_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        d = abs(row1[i] - row2[i])
        if d > distance:
            distance = d
    return distance

def accuracy(ypred, ytest):
    match = sum(int(a==b) for a,b in zip(ypred, ytest))
    return match/len(ypred)

class SimpleKNN:
    def __init__(self, k, distance):
        self.distf = distance
        self.k = k 
        
    def predict(self, x_train, y_train, x_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        res = list()
        for row in x_test:
            neighbors = self.get_neighbors(row)
            prediction = self.make_prediction(neighbors)
            res.append(prediction)
        return res
            
    def get_neighbors(self, test_row):
        d = list()
        n = list()
        for row, c in zip(self.x_train, self.y_train):
            dist = self.distf(row, test_row)
            d.append((row,dist,c))
        d.sort(key=lambda a: a[1])
        for i in range(self.k):
            n.append(d[i])
        return n
            
        
    def make_prediction(self, neighbors):
        classes = [row[-1] for row in neighbors]
        return max(set(classes), key=classes.count)
        

In [3]:
from sklearn.datasets import load_digits

digits = load_digits()
print(digits.data.shape)

(1797, 64)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, test_size=0.7, shuffle=False
)

## Evaluation

In [69]:
def perform_test(max_k, dist_func):
    for i in range(1, max_k + 1):
        predictor = SimpleKNN(i, dist_func)
        ypred = predictor.predict(X_train, y_train, X_test)
        print("i: ", i, " accuracy: ", accuracy(ypred, y_test))

In [70]:
perform_test(10, euclidean_distance)

i:  1  accuracy:  0.9451510333863276
i:  2  accuracy:  0.9324324324324325
i:  3  accuracy:  0.9435612082670907
i:  4  accuracy:  0.9427662957074722
i:  5  accuracy:  0.93879173290938
i:  6  accuracy:  0.9348171701112877
i:  7  accuracy:  0.9332273449920508
i:  8  accuracy:  0.9260731319554849
i:  9  accuracy:  0.9236883942766295
i:  10  accuracy:  0.9173290937996821


In [71]:
perform_test(10, euclidean_squared)

i:  1  accuracy:  0.9451510333863276
i:  2  accuracy:  0.9324324324324325
i:  3  accuracy:  0.9435612082670907
i:  4  accuracy:  0.9427662957074722
i:  5  accuracy:  0.93879173290938
i:  6  accuracy:  0.9348171701112877
i:  7  accuracy:  0.9332273449920508
i:  8  accuracy:  0.9260731319554849
i:  9  accuracy:  0.9236883942766295
i:  10  accuracy:  0.9173290937996821


In [72]:
perform_test(10, metric_distance)

i:  1  accuracy:  0.9395866454689984
i:  2  accuracy:  0.9300476947535771
i:  3  accuracy:  0.9443561208267091
i:  4  accuracy:  0.9411764705882353
i:  5  accuracy:  0.9403815580286169
i:  6  accuracy:  0.9364069952305246
i:  7  accuracy:  0.9348171701112877
i:  8  accuracy:  0.9260731319554849
i:  9  accuracy:  0.9220985691573926
i:  10  accuracy:  0.9165341812400636


In [75]:
perform_test(10, cheb_distance)

i:  1  accuracy:  0.9181240063593005
i:  2  accuracy:  0.8910969793322735
i:  3  accuracy:  0.9133545310015898
i:  4  accuracy:  0.8982511923688394
i:  5  accuracy:  0.9054054054054054
i:  6  accuracy:  0.8942766295707473
i:  7  accuracy:  0.8982511923688394
i:  8  accuracy:  0.8926868044515104
i:  9  accuracy:  0.8990461049284578
i:  10  accuracy:  0.890302066772655


## Noise

In [77]:
from random import randint
import numpy as np

def additive_noise(rows, delta):
    res = rows.copy()
    for row in res:
        for i in row:
            i = np.clip(i + randint(-1*delta,delta), 0, 255)
    return res

In [78]:
noised = additive_noise(X_train[:100], 5)

In [82]:
predictor = SimpleKNN(5, euclidean_distance)
ypred = predictor.predict(X_train, y_train, noised)
print("accuracy: ", accuracy(ypred, y_train[:100]))

accuracy:  0.99


## ODIN